In [33]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(subscription_id= "6b4af8be-9931-443e-90f6-c4c34a1f9737",name="udacity-project", resource_group = "aml-quickstarts-141029")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")
env= Environment.get(workspace=ws, name="AzureML-Minimal")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + "aml-quickstarts-141029")

run = exp.start_logging()

Workspace name: quick-starts-ws-141029 Azure region: southcentralus Subscription id: 6b4af8be-9931-443e-90f6-c4c34a1f9737 Resource group: aml-quickstarts-141029


In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cpu_cluster_name= "udacity-project"
compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

In [26]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
# we are looking for a discrete variable, "yes" or "no"
# choose random sweep for efficiency; faster than grid search and equally effective. 
# Bayesian is out of the question, since it does not support an early termination policy

ps = RandomParameterSampling(parameter_space=  {"C":uniform(0.1,100)})
# Specify an early termination Policy. Early termination improves computational efficiency.
# Bandit is an aggresive policy, saving a lot of time but mabe missing some promising candidates

policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

## for good measure, add a deadline. Cut off after 100 minutes (we only have 120 Minutes free for our Remote session)

max_time = 100

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

##### ????
from azureml.core import ScriptRunConfig, Experiment, Environment
#env = Environment.get(ws)
# configure and submit your training run
config = ScriptRunConfig(source_directory='.',
                            command=['python', 'train.py'],
                            compute_target=cpu_cluster,
                            #environment=env)
                            )
script_run = exp.submit(config)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=config,
                             hyperparameter_sampling=ps,
                             policy= policy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_duration_minutes = max_time,
                             max_total_runs=100,
                             max_concurrent_runs=4)


In [29]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment
#experiment = Experiment(ws, exp)
hyperdrive_run = exp.submit(hyperdrive_config)

# visualize with widget
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [30]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url_paths = [
            'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
            ]

ds = Dataset.File.from_files(path=url_paths)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='spearman_correlation',
    training_data=X_train,
    label_column_name="y",
    n_cross_validations=)

In [2]:
# Submit your automl run

experiment = Experiment(ws, "automl_run")
local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)